### Simple `Encoder-Decoder` model.

In the previous notebook  we covered much on how to create a simple `Bidirectional RNN` model that learns to translate text from english to french. In this notebook we are going to expand this even futher by make use of the `Encoder-Decoder` achitecture to perform the same task.

### Encoder-Decoder

This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.

> Let's jump into the code

**Note**: The rest of the notebook will remain the same, when there's a change i will highlight.

### Imports

In [6]:
from collections import Counter
import numpy as np
import helper, os, time

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.__version__

'2.5.0'

### Mounting the Google Drive.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Paths to the files

In [3]:
base_path = '/content/drive/MyDrive/NLP Data/seq2seq/fr-en-small'
en_path = 'small_vocab_en.txt'
fr_path = 'small_vocab_fr.txt'

### Loading the data.

We have two files that are located at this path `'/content/drive/MyDrive/NLP Data/seq2seq/fr-en-small'` and thes files are:

```
small_vocab_fr.txt
small_vocab_en.txt
```

The following line help us to load the data.

In [10]:
eng_sents = open(os.path.join(base_path, en_path), encoding='utf8').read().split('\n')
fre_sents = open(os.path.join(base_path, fr_path), encoding='utf8').read().split('\n')

print("Data Loaded")

Data Loaded


In [11]:
eng_sents[1]

'the united states is usually chilly during july , and it is usually freezing in november .'

By looking at the data we can see that the data is already preprocessed, which means we are not going to do that step here.

### Next, Bulding the Vocabulary.

Vocabulary in my definition is just unique words in the curpus. Let's look at the vocabulary size of french and english. But first we need to tokenize each sentence, Inorder for us to do that I'm going to use the `spacy` library which is my favourite when it comes to tokenization of languages.

In [12]:
import spacy
spacy.cli.download('fr_core_news_sm')

spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')


✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [13]:
def tokenize_fr(sent):
  return [tok.text for tok in spacy_fr.tokenizer(sent)]
  
def tokenize_en(sent):
  return [tok.text for tok in spacy_en.tokenizer(sent)]

In [14]:
en_counter = Counter()
fr_counter = Counter()

for sent in eng_sents:
  en_counter.update(tokenize_en(sent.lower()))
for sent in fre_sents:
  fr_counter.update(tokenize_fr(sent.lower()))

In [15]:
en_vocab_size = len(en_counter)
fr_vocab_size = len(fr_counter)

fr_vocab_size, en_vocab_size

(340, 201)

Here we have `340` unique words for french in this dataset and `201` unique words for english.

### Preprocessing.

We will convert our text data into sequence of integers so basically we are going to perform the following:

1. Tokenize the words into ids
2. Pad the tokens so that they will have same length.

For this task we are going to use the keras `Tokenizer` class to perform the task, We have been using this for sentiment analyisis so the procedure is the same.

We are going to have two tokenizers for each language.


In [16]:
en_tokenizer = Tokenizer(num_words=en_vocab_size, oov_token="<oov>")
en_tokenizer.fit_on_texts(eng_sents)

fr_tokenizer = Tokenizer(num_words=fr_vocab_size, oov_token="<oov>")
fr_tokenizer.fit_on_texts(fre_sents)

In [17]:
en_word_indices = en_tokenizer.word_index
en_word_indices_reversed = dict([
    (v, k) for (k, v) in en_word_indices.items()
])

fr_word_indices = fr_tokenizer.word_index
fr_word_indices_reversed = dict([
    (v, k) for (k, v) in fr_word_indices.items()
])

### Helper functions
We will create some helper function that converts sequences to text and text to sequences for each language. These function will be used for inference later on.

**We have set the out of vocabulary `oov_token|| <"oov">`token to `1`  which means the word that does not exist in the vocabulary it's integer representation is 1**

In [18]:
def en_seq_to_text(sequences):
  return " ".join(en_word_indices_reversed[i] for i in sequences )

def en_seq_to_text(sequences):
  return " ".join(fr_word_indices_reversed[i] for i in sequences )

def en_text_to_seq(sent):
  words = tokenize_en(sent.lower())
  sequences = []
  for word in words:
    try:
      sequences.append(en_word_indices[word])
    except:
      sequences.append(1)
  return sequences

def fr_text_to_seq(sent):
  words = tokenize_fr(sent.lower())
  sequences = []
  for word in words:
    try:
      sequences.append(fr_word_indices[word])
    except:
      sequences.append(1)
  return sequences

### Converting text to sequences

In [19]:
en_sequences = en_tokenizer.texts_to_sequences(eng_sents)
fr_sequences = fr_tokenizer.texts_to_sequences(fre_sents)

In [20]:
fr_sequences[0:4]

[[36, 35, 2, 9, 68, 38, 12, 25, 7, 4, 2, 113, 3, 51],
 [5, 33, 32, 2, 13, 20, 3, 50, 7, 4, 96, 70, 3, 52],
 [102, 2, 13, 68, 3, 46, 7, 4, 2, 13, 22, 3, 42],
 [5, 33, 32, 2, 9, 270, 3, 42, 7, 4, 104, 20, 3, 49]]

### Padding Sequences.

In our case we are going to assume that the longest sentence has `100` words for both `fr` and `en` languages.

In [21]:
max_words = 100
en_tokens_padded = pad_sequences(
    en_sequences, 
    maxlen=max_words, 
    padding="post", 
    truncating="post"
)
fr_tokens_padded = pad_sequences(
    fr_sequences, 
    maxlen=max_words, 
    padding="post", 
    truncating="post"
)

In [22]:
en_tokens_padded[:2]

array([[18, 24,  2,  9, 68,  5, 40,  8,  4,  2, 56,  3, 45,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 6, 21, 22,  2, 10, 63,  5, 44,  8,  4,  2, 10, 52,  3, 46,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

### Logits to text.

We are going to create 1 more helper function that will help us to take logits or the predictions probabilities and then we convert them to human understandable format.

In [23]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id
                    in tokenizer.word_index.items()}
  index_to_words[0] = '<pad>'
  """
  For every prediction we are going to ignore the pad token
  """
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)]).replace("<pad>", "")


### Encoder Decoder Model.
The following cell shows how we can create our very simple encoder decoder model using the sequential API.


In [58]:
input_shape = (None, 100, 1)
model = keras.Sequential([
    keras.layers.Embedding(
        en_vocab_size,
        128, 
        input_length=max_words
    ),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128, return_sequences=False),
    keras.layers.RepeatVector(max_words),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(
        fr_vocab_size, activation= "softmax"
    ))   
], name="encoder_decoder_model")

model.summary()

Model: "encoder_decoder_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 128)          25728     
_________________________________________________________________
gru_21 (GRU)                 (None, 100, 128)          99072     
_________________________________________________________________
gru_22 (GRU)                 (None, 128)               99072     
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 100, 128)          0         
_________________________________________________________________
gru_23 (GRU)                 (None, 100, 128)          99072     
_________________________________________________________________
gru_24 (GRU)                 (None, 100, 128)          99072     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 100, 340)

In [59]:
tmp_x = en_tokens_padded.reshape(
   -1, 100, 1
)
tmp_x.shape

(137861, 100, 1)

In [62]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(tmp_x, 
          fr_tokens_padded, 
          batch_size=1024, 
          epochs=15,
          validation_split=0.2
)


Epoch 1/15
108/108 [==============================] - 35s 287ms/step - loss: 0.3836 - accuracy: 0.9072 - val_loss: 0.3669 - val_accuracy: 0.9099
Epoch 2/15
108/108 [==============================] - 30s 280ms/step - loss: 0.3626 - accuracy: 0.9101 - val_loss: 0.3577 - val_accuracy: 0.9108
Epoch 3/15
108/108 [==============================] - 30s 280ms/step - loss: 0.3543 - accuracy: 0.9110 - val_loss: 0.3497 - val_accuracy: 0.9118
Epoch 4/15
108/108 [==============================] - 30s 280ms/step - loss: 0.3470 - accuracy: 0.9119 - val_loss: 0.3445 - val_accuracy: 0.9123
Epoch 5/15
108/108 [==============================] - 30s 281ms/step - loss: 0.3409 - accuracy: 0.9124 - val_loss: 0.3417 - val_accuracy: 0.9122
Epoch 6/15
108/108 [==============================] - 30s 282ms/step - loss: 0.3315 - accuracy: 0.9136 - val_loss: 0.3213 - val_accuracy: 0.9151
Epoch 7/15
108/108 [==============================] - 30s 282ms/step - loss: 0.2975 - accuracy: 0.9190 - val_loss: 0.2780 - val_ac

The rest of the notebook will remain the same.


### Making some predictions.
Our model is targeting to predict french words, during the predict function we are going to do the following:

1. Get the sequence of the english sentence 
2. Pad the english sequences and pass them to the model'
3. Reshape the logits output to the shape of `(max_len, trg_vocabsize(french)`
4. Call the `logits_to_text` function and pass the tokenizer as the `fr_tokenizer`.
5. Get the predictions

In [63]:
def predict(sent):
  sequences = en_text_to_seq(sent)
  padded_tokens = pad_sequences([sequences], maxlen=max_words, padding="post", truncating="post")
  logits = model(padded_tokens)
  logits = tf.reshape(logits, (100, -1))
  return logits_to_text(logits, fr_tokenizer)
predict("your least liked fruit is the grape.")

"son fruit préféré est l'orange mais la est                                                                                            "

### Making more predictions.


In [64]:
from prettytable import PrettyTable
def tabulate_translations(column_names, data, title, max_characters=25):
  table = PrettyTable(column_names)
  table.title= title
  table.align[column_names[0]] = 'l'
  table.align[column_names[1]] = 'l'
  table.align[column_names[2]] = 'l'
  table._max_width = {column_names[0] :max_characters, column_names[1] :max_characters, column_names[2]:max_characters}
  for row in data:
    table.add_row(row)
  print(table)
columns_names = [
    "English (real src sentence)", "French (the actual text)", "Translated (translated version)"
]
title = "ENGLISH TO FRENCH TRANSLATOR"

In [66]:
max_characters= 25
total_translations= 10
for i, (eng, fre) in enumerate(zip(eng_sents[:total_translations], fre_sents)):
    rows_data = [[eng, fre, predict(eng)]]
    tabulate_translations(columns_names, rows_data, title, max_characters)

+-------------------------------------------------------------------------------------------+
|                                ENGLISH TO FRENCH TRANSLATOR                               |
+-----------------------------+---------------------------+---------------------------------+
| English (real src sentence) | French (the actual text)  | Translated (translated version) |
+-----------------------------+---------------------------+---------------------------------+
| new jersey is sometimes     | new jersey est parfois    | new jersey est jamais chaud au  |
| quiet during autumn , and   | calme pendant l' automne  | l' de il il est agréable en     |
| it is snowy in april .      | , et il est neigeux en    | novembre                        |
|                             | avril .                   |                                 |
+-----------------------------+---------------------------+---------------------------------+
+-----------------------------------------------------------

### Conclusion.
In this notebook we have learnt how to create a simple Encoder Decoder model and we trained our model and get a resonable accuracy of `~93%`. What's next?

### Next
In the next notebook we will combine all these three notebooks by creating a `spanish-to-english` translator model.